# BAU Forecasting
Forecast each province according to business-as-usual housing growth (aligning with the avg % of housing built in the last ten year)

In [1]:
# Import Libraries
# Data
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
import geopandas as gpd
import random

import re
from copy import deepcopy
#from tqdm import tqdm
import tqdm.notebook as tq

#Viz
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(font='Helvetica') # Futura? Calibri Light? 
sns.set_style("white")
sns.set_theme(style='ticks')
sns.set_context('talk')

from datetime import datetime

c:\Users\Keagan Rankin\anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
from fig_package.forecast import *
from fig_package.fig_project.project_starts import *
from fig_package.fig_sample.road_clean_sample import *
from fig_package.fig_helper.helper import *
from fig_package.fig_sample.house_clean_sample import *

### Inits
Run some inits for variables, including an initlization of the % of each house to be built in the BAU case for each province.

In [3]:
# Init full province loop variables: name, infill rate
abbrevation_name_list = ['nl','ns','nb','pei','qc','on','mb','sk','ab','bc',
                         ]
full_name_list = ['newfoundland','nova_scotia','new_brunswick','pei','quebec','ontario',
                  'manitoba','saskatchewan','alberta','british_columbia',
                  ]
infill_ps = [0.175, 0.175, 0.175, 0.175, 0.35, 0.35, 0.175, 0.175, 0.175, 0.35]

# from 2021 census: low-rise high rise breakdown for each province in order
lr = [12325, 64575, 51985, 10370, 1242910, 0.35, 74865, 61885, 247030, 417475]
hr = [810, 28650, 4260, 130, 225745, 0.65, 43665, 11040, 74880, 221845]

hr[5]/(lr[5]+hr[5])+0.1

0.75

In [4]:
# Need to initialize the percentage of each general housing form (single-family, missing middle, mid-high rise)
# to be built in each province -> ten-year historical averages like with Ontario
# from StatsCan: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3410013501
# create a function to calculate the avg % of each housing type built in the last 10 years
path_hist = "C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Project 2. Housing Projections/FIG_Canada/data/houses/historic_starts_by_prov_for_bau/"

def percent_former(path, abbreviation, lr, hr):
    """return 10-year avg % of forms built in each prov."""
    # read in statscan data and format it.
    forms_sc = pd.read_csv(path+abbreviation+'_historic_starts.csv', skiprows=lambda x: (x <= 9) | (x>17), sep=',',thousands=',').drop(0).drop("Housing estimates", axis=1)
    forms_sc['Q1 2013'] = forms_sc['Q1 2013'].str.replace(',','').astype(float)
    forms_sc.columns = ['type']+[x[3:] for x in forms_sc.columns[1:]]

    # drop unneeded 2023 col and multiples row. split apartments
    forms_sc = forms_sc.groupby(forms_sc.columns, axis=1).sum().drop('2023', axis=1).drop(3).set_index('type')

    # split apartment into lowrise and high rise. Split number based on census breakdown
    forms_sc.loc['lr_apartment'] = forms_sc.loc['Apartment and other unit type']*(lr/(lr+hr)-0.1)
    forms_sc.loc['hr_apartment'] = forms_sc.loc['Apartment and other unit type']*(hr/(lr+hr)+0.1)

    # determine percentages
    sf_p = forms_sc.loc['Single-detached'].sum()/forms_sc.loc['Total units'].sum()
    mm_p = forms_sc.loc[['Semi-detached','Row','lr_apartment']].sum().sum()/forms_sc.loc['Total units'].sum()
    hr_p = forms_sc.loc['hr_apartment'].sum()/forms_sc.loc['Total units'].sum()

    ps = [sf_p.mean().round(2), mm_p.mean().round(2), hr_p.mean().round(2)]
    return ps

m = []
i=0
for ab in abbrevation_name_list:
    #print(ab, percent_former(path_hist, ab))
    m.append(percent_former(path_hist, ab, lr[i], hr[i]))
    i+=1

In [5]:
bau_percentages_df = pd.DataFrame(data=m, columns=['sf','mm','hr'])
bau_percentages_df['prov'] = abbrevation_name_list
bau_percentages_df.set_index('prov')

,sf,mm,hr
prov,,,
nl,0.75,0.23,0.02
ns,0.40,0.40,0.20
nb,0.42,0.51,0.07
pei,0.47,0.49,0.03
qc,0.24,0.59,0.17
on,0.33,0.32,0.35
mb,0.44,0.38,0.19
sk,0.49,0.43,0.09
ab,0.44,0.45,0.11


### Running the forecast
we want to run each housing form seperately for each province, breaking down the amount built by the BAU percentages provided in bau_percentages_df

In [12]:
s_starts[0].loc[2029:2030][name+'_d_add']

Year
2029    169440.389398
2030    172777.413405
Name: british_columbia_d_add, dtype: float64

The forecast only has to be run for 2022 to fix the graph. I will see if I can do a BAU run for the 1 year and keep within the percentages

In [13]:
# run full country loop
full_short_term = []
for i, name in enumerate(full_name_list):
    # init
    print(name)
    forecaster = Forecast()
    forecaster.province_name = name

    # import prep sample
    p = 'C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Project 2. Housing Projections/FIG_Canada/mc_samples/'
    mc = pd.read_csv(p + abbrevation_name_list[i] + '/inf_mc_samples_' + abbrevation_name_list[i] + '.csv')
    # LOOKS LIKE PREPPING WITH IMPORT DOES NOT WORK HERE BECAUSE i NEED THE PROV DA FOR THE BAU SAMPLING
    # import house data
    hs = HouseSample(master=forecaster.master)
    hs.province_name = forecaster.province_name
    prov_da = hs.get_prov_da(prov_name=forecaster.province_name, shapefile_path=hs.shapefile_path, 
                                census_data_path=hs.path + 'da_census_data_reduced/' + hs.prov_da_file_map[forecaster.province_name], dropna=True)

    # Import raod data
    rc = RoadClean(master=forecaster.master)
    rc.province_name = forecaster.province_name
    roads_clean = rc.full_road_clean_map()
    road_filt = roads_clean.groupby('DAUID').agg({'LENGTH_GEO':np.sum})
    handy = Helper()
    road_filt = handy.drop_outliers_iqr(road_filt, 'LENGTH_GEO', f=3)

    samps_p = forecaster.prep_sample(samp=mc, n_iters=500, prov_da=prov_da, road_filter_index=road_filt.index,
                                    sf=[0.0, 1.0], mm=[0.0, 1.0], hr=[0.0, 1.0])

    # get projected starts
    p_starter = ProjectStarts()
    s_starts = p_starter.provincial_starts(name)
    # transform to just the base year, run d and d_add: 
    starts_uno = s_starts[0].loc[2029:2030][name+'_d_add']

    # run generic matflow - change '_d_add' to '_d' to  do BAU vs. high growth
    prov_f = forecaster.matflow_bau(it=100, samps_p=samps_p, starts=np.array(starts_uno), prov_da=prov_da, percent_vector=bau_percentages_df.loc[i][:-1],
                                    infill_percent=infill_ps[i])
    
    full_short_term.append(prov_f)

newfoundland
[I] Reading DA Shapefile...
[I] Importing census data...
[I] Merging data...
[I] Adding useful columns...
[C] Returning... Complete.
Cleaning road data...

Import-clean complete. Returning.
dropped:  55


c:\Users\Keagan Rankin\OneDrive - University of Toronto\Saxe - Rankin\Project 2. Housing Projections\FIG_Canada\model\fig_package\fig_project\project_starts.py:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  self.pop_proj = pd.read_csv(self.path+'canada population projections/17100057.csv')


  0%|          | 0/2 [00:00<?, ?it/s]

year:  0
building new...
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
building iter:  10
building iter:  11
building iter:  12
building iter:  13
building iter:  14
building iter:  15
building iter:  16
building iter:  17
building iter:  18
building iter:  19
building iter:  20
building iter:  21
building iter:  22
building iter:  23
building iter:  24
building iter:  25
building iter:  26
building iter:  27
building iter:  28
building iter:  29
building iter:  30
building iter:  31
building iter:  32
building iter:  33
building iter:  34
building iter:  35
building iter:  36
building iter:  37
building iter:  38
building iter:  39
building iter:  40
building iter:  41
building iter:  42
building iter:  43
building iter:  44
building iter:  45
building iter:  46
building iter:  47
building iter:  48
building iter:  49
building iter:  50
building iter:  

c:\Users\Keagan Rankin\OneDrive - University of Toronto\Saxe - Rankin\Project 2. Housing Projections\FIG_Canada\model\fig_package\fig_sample\road_clean_sample.py:50: DtypeWarning: Columns (10,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  province = pd.read_csv(self.path+road_file_name_map[province_name])


...

  0%|          | 0/2 [00:00<?, ?it/s]

year:  0
building new...
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
building iter:  10
building iter:  11
building iter:  12
building iter:  13
building iter:  14
building iter:  15
building iter:  16
building iter:  17
building iter:  18
building iter:  19
building iter:  20
building iter:  21
building iter:  22
building iter:  23
building iter:  24
building iter:  25
building iter:  26
building iter:  27
building iter:  28
building iter:  29
building iter:  30
building iter:  31
building iter:  32
building iter:  33
building iter:  34
building iter:  35
building iter:  36
building iter:  37
building iter:  38
building iter:  39
building iter:  40
building iter:  41
building iter:  42
building iter:  43
building iter:  44
building iter:  45
building iter:  46
building iter:  47
building iter:  48
building iter:  49
building iter:  50
building iter:  

c:\Users\Keagan Rankin\OneDrive - University of Toronto\Saxe - Rankin\Project 2. Housing Projections\FIG_Canada\model\fig_package\fig_project\project_starts.py:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  self.pop_proj = pd.read_csv(self.path+'canada population projections/17100057.csv')


  0%|          | 0/2 [00:00<?, ?it/s]

year:  0
building new...
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
building iter:  10
building iter:  11
building iter:  12
building iter:  13
building iter:  14
building iter:  15
building iter:  16
building iter:  17
building iter:  18
building iter:  19
building iter:  20
building iter:  21
building iter:  22
building iter:  23
building iter:  24
building iter:  25
building iter:  26
building iter:  27
building iter:  28
building iter:  29
building iter:  30
building iter:  31
building iter:  32
building iter:  33
building iter:  34
building iter:  35
building iter:  36
building iter:  37
building iter:  38
building iter:  39
building iter:  40
building iter:  41
building iter:  42
building iter:  43
building iter:  44
building iter:  45
building iter:  46
building iter:  47
building iter:  48
building iter:  49
building iter:  50
building iter:  

In [14]:
# Save results
full_inter = []
for full in full_short_term:
    full_inter.append(pd.concat(full))

In [15]:
pd.concat(full_inter)

,DAUID,house_ghg_pp,road_ghg_pp,water_inf_ghg_pp,iter,tot_pd_check_count,single_detached,missing_middle,mid_high_rise,year
59173,10070529,36789.792041,15191.387740,1037.027374,0,25.0,25.0,0.0,0.0,0
74252,10010572,23097.107464,357.827684,320.918017,0,620.0,420.0,195.0,5.0,0
160775,10090095,27965.217364,7309.396672,175.491558,0,165.0,165.0,5.0,0.0,0
405880,10060095,24603.003875,3702.725330,321.411418,0,155.0,145.0,10.0,0.0,0
365799,10070531,31468.400442,2974.723483,335.553023,0,215.0,170.0,45.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
1711681,59151338,10807.045263,0.000000,0.000000,99,1150.0,20.0,150.0,980.0,1
1514383,59150822,12872.025524,0.000000,0.000000,99,235.0,0.0,115.0,125.0,1
771791,59150961,16848.294046,0.000000,0.000000,99,180.0,120.0,60.0,0.0,1
540952,59152832,12496.644355,0.000000,0.000000,99,1060.0,5.0,80.0,980.0,1


In [16]:
p = 'C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Project 2. Housing Projections/FIG_Canada/results/bau/'
pd.concat(full_inter).set_index('DAUID').to_csv(p+'bau_add_d_2029_2030.csv')

checking a single iteration

In [8]:
# init
name = full_name_list[0]
forecaster = Forecast()
forecaster.province_name = name

# import prep sample
p = 'C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Project 2. Housing Projections/FIG_Canada/mc_samples/'
mc = pd.read_csv(p + abbrevation_name_list[0] + '/inf_mc_samples_' + abbrevation_name_list[0] + '.csv')
# LOOKS LIKE PREPPING WITH IMPORT DOES NOT WORK HERE BECAUSE i NEED THE PROV DA FOR THE BAU SAMPLING
# import house data
hs = HouseSample(master=forecaster.master)
hs.province_name = forecaster.province_name
prov_da = hs.get_prov_da(prov_name=forecaster.province_name, shapefile_path=hs.shapefile_path, 
                            census_data_path=hs.path + 'da_census_data_reduced/' + hs.prov_da_file_map[forecaster.province_name], dropna=True)

# Import raod data
rc = RoadClean(master=forecaster.master)
rc.province_name = forecaster.province_name
roads_clean = rc.full_road_clean_map()
road_filt = roads_clean.groupby('DAUID').agg({'LENGTH_GEO':np.sum})
handy = Helper()
road_filt = handy.drop_outliers_iqr(road_filt, 'LENGTH_GEO', f=3)

samps_p = forecaster.prep_sample(samp=mc, n_iters=500, prov_da=prov_da, road_filter_index=road_filt.index,
                                 sf=[0.0, 1.0], mm=[0.0, 1.0], hr=[0.0, 1.0])

# get projected starts
p_starter = ProjectStarts()
s_starts = p_starter.provincial_starts(name)
# transform to just the year 2022
starts_uno = s_starts[0].loc[2022:2023][name+'_d']

# run generic matflow - change '_d_add' to '_d' to  do BAU vs. high growth
prov_f = forecaster.matflow_bau(it=10, samps_p=samps_p, starts=np.array(starts_uno), prov_da=prov_da, percent_vector=bau_percentages_df.loc[0][:-1],
                                infill_percent=infill_ps[0])

[I] Reading DA Shapefile...
[I] Importing census data...
[I] Merging data...
[I] Adding useful columns...
[C] Returning... Complete.
Cleaning road data...

Import-clean complete. Returning.
dropped:  55


c:\Users\Keagan Rankin\OneDrive - University of Toronto\Saxe - Rankin\Project 2. Housing Projections\FIG_Canada\model\fig_package\fig_project\project_starts.py:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  self.pop_proj = pd.read_csv(self.path+'canada population projections/17100057.csv')


  0%|          | 0/2 [00:00<?, ?it/s]

year:  0
building new...
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
infilling...
matflow it: 10
infill func it: 10
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
year:  1
building new...
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9
infilling...
matflow it: 10
infill func it: 10
building iter:  0
building iter:  1
building iter:  2
building iter:  3
building iter:  4
building iter:  5
building iter:  6
building iter:  7
building iter:  8
building iter:  9


In [19]:
k=1
print('sf', prov_f[k]['single_detached'].sum()/prov_f[k]['tot_pd_check_count'].sum())
print('mm', prov_f[k]['missing_middle'].sum()/prov_f[k]['tot_pd_check_count'].sum())
print('hr', prov_f[k]['mid_high_rise'].sum()/prov_f[k]['tot_pd_check_count'].sum())

sf 0.6683673469387755
mm 0.31746031746031744
hr 0.008503401360544218
